In [ ]:
import numpy as np
from scipy import stats as st
import seaborn as sns
import panel as pn
from matplotlib.figure import Figure

pn.extension('ipywidgets')

In [ ]:
def get_d6_hist(log_n_samples=0, kde=False):
    """Plots histogram of a D6"""
    
    fig = Figure(figsize=(6, 6))
    ax = fig.subplots()

    n_samples = int(10**log_n_samples)
    samples = np.floor(st.uniform(0, 6).rvs(n_samples)).astype(int) + 1
    sns.histplot(samples, bins=range(1, 8), ax=ax, kde=kde, stat="probability")
    ax.set_title(f"Histogram of a D6 with {n_samples} samples\nEstimated Mean: {samples.mean():.2f}")
    return fig

n_samples_widget = pn.widgets.FloatSlider(name="Log base 10 of the number of samples", value=0, start=0, end=5)
rv_plot = pn.bind(get_d6_hist, log_n_samples=n_samples_widget)

rv_pane = pn.Column(n_samples_widget, rv_plot);

In [ ]:
st.uniform(0, 6).ppf(.05)

In [ ]:
def get_two_dist_hist(log_n_samples=0, kde=False):
    """Plots histogram of a D6"""
    
    fig = Figure(figsize=(6, 6))
    ax = fig.subplots()

    n_samples = int(10**log_n_samples)
    def _gen_samples(loc, scale, n_samples):
        return np.floor(st.uniform(loc, scale).rvs(n_samples)).astype(int)
    samples_x =_gen_samples(1, 6, n_samples)
    samples_y = np.concatenate([_gen_samples(1, 4, n_samples), _gen_samples(3, 4, n_samples)])
    sns.histplot(samples_x, bins=range(1, 8), ax=ax, kde=kde, stat="probability")
    sns.histplot(samples_y, bins=range(1, 8), ax=ax, kde=kde, stat="probability")
    ax.set_title(f"Histogram of a X (D6) and Y (D3 + 3) with {n_samples} samples\nEstimated Mean of X: {samples_x.mean():.2f} and Y: {samples_y.mean():.2f}")
    return fig

dist_plot = pn.bind(get_two_dist_hist, log_n_samples=n_samples_widget)

dist_pane = pn.Column(n_samples_widget, dist_plot)

In [ ]:
pn.Tabs(
    ('RV', rv_pane),######wsasadasdsa asdadsasd
    ('Two Dists', dist_pane),
    dynamic=True, sizing_mode='stretch_both'
).servable()
